In [1]:
import os
import numpy as np
import dask.dataframe as dd
import pandas as pd
import concurrent.futures
import warnings
import timeit
import time
warnings.filterwarnings("ignore")
import re
import matching
from geopy import distance

In [2]:
df = pd.read_excel('20210702_pb_data_postcode_1.xlsx')

In [2]:
df = pd.read_csv('ZP_RM_61k_with_address_base_ofcom_added.csv')

In [3]:
dff = pd.read_csv('schools.csv')

In [4]:
mp = {}

In [36]:
dff[pd.isnull(dff['Postcode'])]

,UUID,URN,LA_CODE,LA_NAME,Establishment_Number,Establishment_Name,Type_Of_Establishment_Code,Type_Of_Establishment_Name,Establishment_Type_Group_Code,Establishment_Type_Group_Name,...,QAB_Name_Code,QAB_Name,Establishment_Accredited_Code,Establishment_Accredited_Name,QAB_Report,CH_Number,MSOA_Code,LSOA_Code,Latitude,Longitude


In [28]:
ans = 0
mx = 0
for x in df['POSTCODE_LOCATOR']:
    if x.split()[0] in mp.keys():
        k = len(mp[x.split()[0]])
        if k>mx:
            mx = k
print(mx)

88


In [5]:
for x in range(len(dff.index)):
    pcode = dff.iloc[x]['Postcode'].split()[0]
    if pcode in mp:
        mp[pcode].append(x)
    else:
        mp[pcode] = [x]
    if x%10000 == 0:
        print(x)

0
10000
20000
30000
40000


In [6]:
def dist(coords_1,coords_2):
    return distance.distance(coords_1, coords_2).km

In [7]:
from math import sin, cos, sqrt, atan2, radians
def my_dist(coords_1,coords_2):
    
    R = 6373.0

    lat1 = radians(coords_1[0])
    lon1 = radians(coords_1[1])
    lat2 = radians(coords_2[0])
    lon2 = radians(coords_2[1])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c

    return distance

In [8]:
dff.columns

Index(['UUID', 'URN', 'LA_CODE', 'LA_NAME', 'Establishment_Number',
       'Establishment_Name', 'Type_Of_Establishment_Code',
       'Type_Of_Establishment_Name', 'Establishment_Type_Group_Code',
       'Establishment_Type_Group_Name',
       ...
       'QAB_Name_Code', 'QAB_Name', 'Establishment_Accredited_Code',
       'Establishment_Accredited_Name', 'QAB_Report', 'CH_Number', 'MSOA_Code',
       'LSOA_Code', 'Latitude', 'Longitude'],
      dtype='object', length=142)

In [8]:
df['school_dist_0'] = 10000000
df['school_URN_0'] = '-'
df['school_Establishment_Name_0'] = '-'
df['school_dist_1'] = 10000000
df['school_URN_1'] = '-'
df['school_Establishment_Name_1'] = '-'
df['school_dist_2'] = 10000000
df['school_URN_2'] = '-'
df['school_Establishment_Name_2'] = '-'

In [9]:
dff

,UUID,URN,LA_CODE,LA_NAME,Establishment_Number,Establishment_Name,Type_Of_Establishment_Code,Type_Of_Establishment_Name,Establishment_Type_Group_Code,Establishment_Type_Group_Name,...,QAB_Name_Code,QAB_Name,Establishment_Accredited_Code,Establishment_Accredited_Name,QAB_Report,CH_Number,MSOA_Code,LSOA_Code,Latitude,Longitude
0,0000b2534da4973cacdaafb14b6a92da,119812,888,Lancashire,5403.0,Clitheroe Royal Grammar School,5,Foundation school,4,Local authority maintained schools,...,0.0,Not applicable,0.0,Not applicable,NaN,NaN,E02005271,E01025344,53.874688,-2.388507
1,000153ba9529e7ce7d116ebccf4959a0,100506,207,Kensington and Chelsea,5402.0,The Cardinal Vaughan Memorial RC School,2,Voluntary aided school,4,Local authority maintained schools,...,0.0,0,0.0,NaN,NaN,0.0,E01002869,51.5038735,-0.211792,NaN
2,0001e2f6248b619be975be631c758d00,111429,896,Cheshire West and Chester,4167.0,The Whitby High School,5,Foundation school,4,Local authority maintained schools,...,0.0,Not applicable,0.0,Not applicable,NaN,NaN,E02003852,E01018526,53.263607,-2.912076
3,00037886c0515afe4c454b285973847e,125933,938,West Sussex,2206.0,Hassocks Infant School,1,Community school,4,Local authority maintained schools,...,0.0,Not applicable,0.0,Not applicable,NaN,NaN,E02006620,E01031746,50.926238,-0.142331
4,00037edc9cb695a2e8a04a0d52cbdb77,140487,861,Stoke-on-Trent,6025.0,Nisai House Stoke,11,Other independent school,3,Independent schools,...,0.0,Not applicable,0.0,Not applicable,NaN,NaN,E02002959,E01014251,53.045348,-2.196612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45027,fffc84600890fe9eb63f011fdd8be079,138669,320,Waltham Forest,6003.0,Alpha Preparatory School,11,Other independent school,3,Independent schools,...,0.0,0,0.0,NaN,NaN,0.0,E01004464,51.5865385,-0.004675,NaN
45028,fffca74a44903a04857e641edf15d174,145978,888,Lancashire,2006.0,Casterton Primary Academy,28,Academy sponsor led,10,Academies,...,0.0,Not applicable,0.0,Not applicable,NaN,NaN,E02005176,E01024891,53.811908,-2.227036
45029,fffcc805733020992991265f86bd36c8,124270,860,Staffordshire,3100.0,All Saints CofE (C) Primary School,3,Voluntary controlled school,4,Local authority maintained schools,...,0.0,Not applicable,0.0,Not applicable,NaN,NaN,E02006145,E01029440,52.803606,-1.732575
45030,fffe8ba956fde53b03c97c112b6ea885,103380,330,Birmingham,2453.0,Leigh Junior Infant and Nursery School,1,Community school,4,Local authority maintained schools,...,0.0,Not applicable,0.0,Not applicable,NaN,NaN,E02001874,E01009479,52.497683,-1.847943


In [10]:
for x in range(len(df.index)):
    pcode = df.iloc[x]['POSTCODE_LOCATOR'].split()[0]
    if pcode not in mp.keys():
        continue
    cand = mp[pcode]
    
    x2 = df.iloc[x]['LATITUDE']
    y2 = df.iloc[x]['LONGITUDE']
    lsto = []
    for i in cand:
        
        x1 = dff.iloc[i]['Latitude']
        y1 = dff.iloc[i]['Longitude']
        
        d = my_dist((x1,y1),(x2,y2))
        lsto.append((d,i))
    
    lsto.sort()
    for i in range(min(len(lsto),3)):
        df.loc[x,'school_dist_{}'.format(i)] = lsto[i][0]
        df.loc[x,'school_URN_{}'.format(i)] = dff.iloc[lsto[i][1]]['URN']
        df.loc[x,'school_Establishment_Name_{}'.format(i)] = dff.iloc[lsto[i][1]]['Establishment_Name']
    
    if x%1000 == 0:
        print(x)
#         if x%10000 == 0:
#             df.to_csv('near_schools_result_2.csv')

            

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000


In [11]:
df

,Unnamed: 0,Date,Property type,Price,Source,Address,KeyFeatures,latitude,longitude,Bedrooms,...,ufbb_availability,school_dist_0,school_URN_0,school_Establishment_Name_0,school_dist_1,school_URN_1,school_Establishment_Name_1,school_dist_2,school_URN_2,school_Establishment_Name_2
0,0,25 Jun 2021,"Detached, Freehold","£995,000",Land Registry,"41 Runcie Close, St Albans, Hertfordshire AL4 9AX",Detached\nFour Bedrooms\nTwo Bathrooms\nStudy\...,51.770175,-0.313172,4,...,100,0.143433,146878,Wheatfields Junior Mixed School,0.143433,146893,Wheatfields Infants' and Nursery School,0.143433,117212,Wheatfields Junior Mixed School
1,1,10 Jun 2021,"Terraced, Freehold","£650,000",Land Registry,"37 Albany Road, Birmingham, West Midlands B17 9JX",VICTORIAN MID TERRACE\nFOUR BEDROOMS\nPERIOD F...,52.459072,-1.950948,4,...,90,0.138282,132261,Harborne Primary School,0.138282,103212,Harborne Infant School,0.138282,103211,Harborne Junior School
2,2,30 Jun 2021,"Detached, Freehold","£350,000",Land Registry,"19 Bibsworth Avenue, Birmingham, West Midlands...",Detached\nThree Bedrooms\nKitchen Diner\nTwo R...,52.432568,-1.859492,3,...,100,0.602562,103462,St Bernard's Catholic Primary School,0.957317,103519,Moseley School and Sixth Form,0.995820,103167,Billesley Infant School
3,3,18 Jun 2021,"Detached, Freehold","£382,500",Land Registry,"24 Sandy Croft, Birmingham, West Midlands B13 0EP",3 Bedrooms\nPorch\nEntrance Hall\nLounge\nKitc...,52.427717,-1.863037,3,...,100,0.473623,103167,Billesley Infant School,0.473623,138888,Billesley Primary School,0.473623,131942,Billesley Primary School
4,4,30 Jul 2021,"Semi-Detached, Freehold","£213,500",Land Registry,"15 Poppy Walk, Hatfield, Hertfordshire AL10 9FP",One Bedroomed House\nAllocated Parking\nChain ...,51.774284,-0.237790,1,...,0,0.276724,117105,Green Lanes Primary School,0.912132,132105,Birchwood Avenue Primary School,0.912132,117104,Gascoyne Cecil Junior School
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61118,61118,Aug 2021,NaN,"£225,000",NaN,53 Strathmore Avenue LU1 3NY,"Terraced house, Freehold, 3 Beds, 2 Baths, 2 R...",51.872773,-0.408768,3.0,...,100,NaN,141014,Education & Youth Services Ltd (Luton),0.342038,134525,Avenue Centre for Education,0.449903,145872,Chiltern Academy
61119,61119,Aug 2021,NaN,"£60,000",NaN,13 Robinson Street DH8 5TR,"Terraced house, Freehold, 2 Beds, 1 Bath, 2 Re...",54.861234,-1.841458,3.0,...,100,NaN,114051,Consett Junior School,0.303556,114295,Consett Community Sports College,0.501237,132886,Benfieldside Junior School
61120,61120,Aug 2021,NaN,"£125,000",NaN,10 Merrington Road TF2 8PY,"Semi-detached house, Freehold",52.725638,-2.419358,2.0,...,100,0.583646,123453,Muxton Primary School,1.422642,123364,Donnington Wood Infant School and Nursery Centre,1.461241,123557,St Matthew's Church of England Aided Primary S...
61121,61121,Aug 2021,NaN,"£318,000",NaN,"8 Mount View, Church Lane West GU11 3LN",NaN,51.243067,-0.767056,NaN,...,92,0.309190,137422,St Joseph's Catholic Primary School,0.309190,116484,St Joseph's Catholic Primary School,0.542398,115836,Rowhill School


In [12]:
df.to_csv('ZP_RM_61k_with_address_base_ofcom_schools_added.csv')